This program assigns roles to team members for the project. 

Conditions:
1. Team will rotate team roles in "cycles".
2. The length of each cycle may be two weeks or more, as determined by the team.
3. Each role has an expert who will guide the team members assigned to the role. 
4. Each team member may be assigned to more than one role at any given time. 
5. No team members, besides the experts, will be assigned to the same role in two consecutive cycles. 
6. Experts may be assigned to other roles just like any other team member. 

In [64]:
import pandas as pd
import numpy as np
import random
random.seed(42)

# Assign Roles 

This part of the code is used to assign roles to the team members from scratch

Instantiate list of members

In [65]:
members = [
            "Oluwaseyi Adaramola",
            "Farook Ajose",
            "Malik Akintola",
            "Ibukun Akinleye",
            "Samson Ilesanmi",
            "Oluwaseunfunmi Alarima",
            "Damilola Omokehinde",
            "Samuel Olayiwola",
            "Adejuwon Adegite",
            "Kafilat Akinyemi",
            "Chibike Enibe",
            "Ayodeji Abatan",
            "Divine Nkwocha",
            "AbdulQahhar Okenla",
        ]

members

['Oluwaseyi Adaramola',
 'Farook Ajose',
 'Malik Akintola',
 'Ibukun Akinleye',
 'Samson Ilesanmi',
 'Oluwaseunfunmi Alarima',
 'Damilola Omokehinde',
 'Samuel Olayiwola',
 'Adejuwon Adegite',
 'Kafilat Akinyemi',
 'Chibike Enibe',
 'Ayodeji Abatan',
 'Divine Nkwocha',
 'AbdulQahhar Okenla']

Instantiate available roles

In [66]:
roles = pd.DataFrame({
    "role": ["Project Manager",
            "Data Analyst (Python)",
            "Data Scientist/Statistician",
            "Visualization Specialist (Power BI)",
            "Technical Writer",
            "Publicity Coordinator",
            "Quality Assurance Engineer"],
    "num_people": [3, 3, 3, 3, 3, 3, 3],
    "expert": ["Farook Ajose",
               "Malik Akintola",
               "Ayodeji Abatan",
               "Samson Ilesanmi",
               "Farook Ajose",
               "AbdulQahhar Okenla",
               "Farook Ajose"]
    })
roles

,role,num_people,expert
0,Project Manager,3,Farook Ajose
1,Data Analyst (Python),3,Malik Akintola
2,Data Scientist/Statistician,3,Ayodeji Abatan
3,Visualization Specialist (Power BI),3,Samson Ilesanmi
4,Technical Writer,3,Farook Ajose
5,Publicity Coordinator,3,AbdulQahhar Okenla
6,Quality Assurance Engineer,3,Farook Ajose


Write function to assign roles

In [67]:
num_cycles = 6

def assign_roles(row, cycles=num_cycles, team=members):
    """Assign team members to different roles for each rotation cycle, ensuring that an expert in the area is present to guide them. 

    Args:
        row (pandas.Series): A row of the `roles` dataframe containing the role details.
        cycles (int, optional): Number of rotation cycles to assign the roles for. Defaults to the value of `num_cycles`.
        team (list, optional): A list of team members to select from. Defaults to the value of `members`.

    Returns:
        expert_cycle_dict (dict): A dictionary containing the assigned members for each cycle, with keys in the format "Cycle x" where x is the cycle number.
    """
    
    # Remove expert from sample list 
    expert = row['expert']
    no_expert_team = team.copy()
    no_expert_team.remove(expert)

    # Create dictionary of assigned members for each cycle
    cycle_dict = {
        f"Cycle {cycle}":  np.random.choice(no_expert_team, size=row["num_people"]-1, replace=False)
        for cycle in range(1, cycles+1)
    }
    
    # Remove common values between consecutive cycles
    for i in range(1, cycles):
        current_cycle = cycle_dict[f"Cycle {i}"]
        next_cycle = cycle_dict[f"Cycle {i+1}"]
        common_members = np.intersect1d(current_cycle, next_cycle)
        for member in common_members:
            new_member = np.random.choice(no_expert_team, size=1)[0]
            while new_member in next_cycle:
                new_member = np.random.choice(no_expert_team, size=1)[0]
            next_cycle = np.where(next_cycle == member, new_member, next_cycle)
        cycle_dict[f"Cycle {i+1}"] = next_cycle
        
    # Append expert to list and convert to string    
    expert_cycle_dict = {
        f"Cycle {cycle}":  ", ".join(np.append(expert, cycle_dict[f"Cycle {cycle}"]))
        for cycle in range(1, cycles+1)
    }

    return expert_cycle_dict
        
# Test function to see outputs.     
assign_roles(roles.iloc[2, :])

{'Cycle 1': 'Ayodeji Abatan, Malik Akintola, Samson Ilesanmi',
 'Cycle 2': 'Ayodeji Abatan, Chibike Enibe, Ibukun Akinleye',
 'Cycle 3': 'Ayodeji Abatan, Kafilat Akinyemi, Divine Nkwocha',
 'Cycle 4': 'Ayodeji Abatan, Damilola Omokehinde, Oluwaseunfunmi Alarima',
 'Cycle 5': 'Ayodeji Abatan, Kafilat Akinyemi, Divine Nkwocha',
 'Cycle 6': 'Ayodeji Abatan, Ibukun Akinleye, Samuel Olayiwola'}

Update dataframe with function

In [68]:
roles[[f"Cycle {cycle}" for cycle in range(1, num_cycles+1)]] = roles.apply(func=assign_roles, axis=1, result_type='expand')
roles

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Project Manager,3,Farook Ajose,"Farook Ajose, Malik Akintola, Kafilat Akinyemi","Farook Ajose, Damilola Omokehinde, Divine Nkwocha","Farook Ajose, Adejuwon Adegite, AbdulQahhar Ok...","Farook Ajose, Ibukun Akinleye, Samson Ilesanmi","Farook Ajose, Divine Nkwocha, Malik Akintola","Farook Ajose, Kafilat Akinyemi, Samuel Olayiwola"
1,Data Analyst (Python),3,Malik Akintola,"Malik Akintola, Divine Nkwocha, Oluwaseunfunmi...","Malik Akintola, Samuel Olayiwola, Ibukun Akinleye","Malik Akintola, Damilola Omokehinde, Adejuwon ...","Malik Akintola, Kafilat Akinyemi, AbdulQahhar ...","Malik Akintola, Samuel Olayiwola, Divine Nkwocha","Malik Akintola, Chibike Enibe, Oluwaseyi Adara..."
2,Data Scientist/Statistician,3,Ayodeji Abatan,"Ayodeji Abatan, Samuel Olayiwola, Chibike Enibe","Ayodeji Abatan, Samuel Olayiwola, Ibukun Akinleye","Ayodeji Abatan, Samson Ilesanmi, AbdulQahhar O...","Ayodeji Abatan, Ibukun Akinleye, Oluwaseunfunm...","Ayodeji Abatan, Divine Nkwocha, Chibike Enibe","Ayodeji Abatan, Oluwaseyi Adaramola, Samuel Ol..."
3,Visualization Specialist (Power BI),3,Samson Ilesanmi,"Samson Ilesanmi, Oluwaseunfunmi Alarima, Malik...","Samson Ilesanmi, Farook Ajose, Ayodeji Abatan","Samson Ilesanmi, Ibukun Akinleye, Malik Akintola","Samson Ilesanmi, Samuel Olayiwola, Oluwaseyi A...","Samson Ilesanmi, Malik Akintola, AbdulQahhar O...","Samson Ilesanmi, Farook Ajose, Divine Nkwocha"
4,Technical Writer,3,Farook Ajose,"Farook Ajose, Ayodeji Abatan, Samuel Olayiwola","Farook Ajose, Oluwaseyi Adaramola, Kafilat Aki...","Farook Ajose, Oluwaseunfunmi Alarima, Adejuwon...","Farook Ajose, Kafilat Akinyemi, Damilola Omoke...","Farook Ajose, Adejuwon Adegite, Samuel Olayiwola","Farook Ajose, Kafilat Akinyemi, Oluwaseyi Adar..."
5,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Farook Ajose, Ayodeji Abatan","AbdulQahhar Okenla, Divine Nkwocha, Samuel Ola...","AbdulQahhar Okenla, Adejuwon Adegite, Ibukun A...","AbdulQahhar Okenla, Samson Ilesanmi, Kafilat A...","AbdulQahhar Okenla, Ayodeji Abatan, Oluwaseyi ...","AbdulQahhar Okenla, Malik Akintola, Oluwaseunf..."
6,Quality Assurance Engineer,3,Farook Ajose,"Farook Ajose, Oluwaseunfunmi Alarima, Samuel O...","Farook Ajose, Ayodeji Abatan, Kafilat Akinyemi","Farook Ajose, Divine Nkwocha, Malik Akintola","Farook Ajose, Oluwaseyi Adaramola, Ibukun Akin...","Farook Ajose, Kafilat Akinyemi, Samuel Olayiwola","Farook Ajose, Malik Akintola, Oluwaseyi Adaramola"


Summarize the roles for each member per cycle

In [69]:
def summarize_roles(df, members=members, cycles=num_cycles):
    role_summary = {}
    for member in members:
        member_roles = []
        for cycle in range(1, cycles+1):
            assigned_roles = ", ".join(df["role"][df[f"Cycle {cycle}"].str.contains(member)])
            member_roles.append(assigned_roles)
        role_summary[member] = member_roles
        
    df = pd.DataFrame(role_summary).T
    df.columns = [f"Cycle {cycle}" for cycle in range(1, cycles+1)]
    
    return df

In [70]:
role_summary = summarize_roles(roles)
role_summary

,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
Oluwaseyi Adaramola,,Technical Writer,,"Visualization Specialist (Power BI), Quality A...",Publicity Coordinator,"Data Analyst (Python), Data Scientist/Statisti..."
Farook Ajose,"Project Manager, Technical Writer, Publicity C...","Project Manager, Visualization Specialist (Pow...","Project Manager, Technical Writer, Quality Ass...","Project Manager, Technical Writer, Quality Ass...","Project Manager, Technical Writer, Quality Ass...","Project Manager, Visualization Specialist (Pow..."
Malik Akintola,"Project Manager, Data Analyst (Python), Visual...",Data Analyst (Python),"Data Analyst (Python), Visualization Specialis...",Data Analyst (Python),"Project Manager, Data Analyst (Python), Visual...","Data Analyst (Python), Publicity Coordinator, ..."
Ibukun Akinleye,,"Data Analyst (Python), Data Scientist/Statisti...","Visualization Specialist (Power BI), Publicity...","Project Manager, Data Scientist/Statistician, ...",,
Samson Ilesanmi,Visualization Specialist (Power BI),Visualization Specialist (Power BI),"Data Scientist/Statistician, Visualization Spe...","Project Manager, Visualization Specialist (Pow...",Visualization Specialist (Power BI),Visualization Specialist (Power BI)
Oluwaseunfunmi Alarima,"Data Analyst (Python), Visualization Specialis...",,Technical Writer,Data Scientist/Statistician,,Publicity Coordinator
Damilola Omokehinde,,Project Manager,Data Analyst (Python),Technical Writer,,
Samuel Olayiwola,"Data Scientist/Statistician, Technical Writer,...","Data Analyst (Python), Data Scientist/Statisti...",,Visualization Specialist (Power BI),"Data Analyst (Python), Technical Writer, Quali...","Project Manager, Data Scientist/Statistician"
Adejuwon Adegite,,,"Project Manager, Data Analyst (Python), Techni...",,Technical Writer,
Kafilat Akinyemi,Project Manager,"Technical Writer, Quality Assurance Engineer",,"Data Analyst (Python), Technical Writer, Publi...",Quality Assurance Engineer,"Project Manager, Technical Writer"


Save dataframe to CSV

In [71]:
# roles.to_csv("assigned_roles.csv", index=False)
# role_summary.to_csv("role_summary.csv")

# Update Existing Roles 

This part of the code is used to update the existing roles in a CSV.

In [72]:
current_roles = pd.read_csv("assigned_roles.csv")
current_roles.head()

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Data Analyst (Python),4,Malik Akintola,"Malik Akintola, Chibike Enibe, AbdulQahhar Oke...","Malik Akintola, Samson Ilesanmi, Kafilat Akiny...","Malik Akintola, Adejuwon Adegite, Chibike Enib...","Malik Akintola, AbdulQahhar Okenla, Farook Ajo...","Malik Akintola, Adejuwon Adegite, Ayodeji Abat...","Malik Akintola, Farook Ajose, Chibike Enibe, D..."
1,Data Scientist/Statistician,4,Ayodeji Abatan,"Ayodeji Abatan, AbdulQahhar Okenla, Divine Nkw...","Ayodeji Abatan, Kafilat Akinyemi, Chibike Enib...","Ayodeji Abatan, Ibukun Akinleye, Oluwaseyi Ada...","Ayodeji Abatan, Oluwaseunfunmi Alarima, Samson...","Ayodeji Abatan, Samuel Olayiwola, Farook Ajose...","Ayodeji Abatan, Malik Akintola, Oluwaseyi Adar..."
2,Visualization Specialist (Power BI),4,Samson Ilesanmi,"Samson Ilesanmi, Ayodeji Abatan, Damilola Omok...","Samson Ilesanmi, Ibukun Akinleye, Farook Ajose...","Samson Ilesanmi, Samuel Olayiwola, Divine Nkwo...","Samson Ilesanmi, Ayodeji Abatan, AbdulQahhar O...","Samson Ilesanmi, Ibukun Akinleye, Oluwaseunfun...","Samson Ilesanmi, AbdulQahhar Okenla, Divine Nk..."
3,Technical Writer,4,Farook Ajose,"Farook Ajose, Oluwaseunfunmi Alarima, AbdulQah...","Farook Ajose, Samson Ilesanmi, Adejuwon Adegit...","Farook Ajose, Chibike Enibe, Samuel Olayiwola,...","Farook Ajose, Oluwaseyi Adaramola, AbdulQahhar...","Farook Ajose, Ibukun Akinleye, Chibike Enibe, ...","Farook Ajose, AbdulQahhar Okenla, Oluwaseyi Ad..."
4,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Ibukun Akinleye, Samuel Ol...","AbdulQahhar Okenla, Damilola Omokehinde, Adeju...","AbdulQahhar Okenla, Malik Akintola, Ibukun Aki...","AbdulQahhar Okenla, Samson Ilesanmi, Adejuwon ...","AbdulQahhar Okenla, Ibukun Akinleye, Kafilat A...","AbdulQahhar Okenla, Oluwaseyi Adaramola, Divin..."


Summarize existing roles

In [73]:
current_role_summary = summarize_roles(current_roles)
current_role_summary

,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
Oluwaseyi Adaramola,Data Analyst (Python),,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,,"Data Scientist/Statistician, Technical Writer,..."
Farook Ajose,Technical Writer,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,"Data Analyst (Python), Technical Writer","Data Scientist/Statistician, Technical Writer","Data Analyst (Python), Technical Writer"
Malik Akintola,"Data Analyst (Python), Data Scientist/Statisti...",Data Analyst (Python),"Data Analyst (Python), Publicity Coordinator","Data Analyst (Python), Visualization Specialis...",Data Analyst (Python),"Data Analyst (Python), Data Scientist/Statisti..."
Ibukun Akinleye,Publicity Coordinator,"Data Analyst (Python), Visualization Specialis...","Data Scientist/Statistician, Technical Writer,...",,"Visualization Specialist (Power BI), Technical...",
Samson Ilesanmi,Visualization Specialist (Power BI),"Data Analyst (Python), Visualization Specialis...",Visualization Specialist (Power BI),"Data Scientist/Statistician, Visualization Spe...","Visualization Specialist (Power BI), Technical...",Visualization Specialist (Power BI)
Oluwaseunfunmi Alarima,Technical Writer,,,Data Scientist/Statistician,Visualization Specialist (Power BI),
Damilola Omokehinde,Visualization Specialist (Power BI),Publicity Coordinator,Data Scientist/Statistician,Data Analyst (Python),Visualization Specialist (Power BI),Data Analyst (Python)
Samuel Olayiwola,Publicity Coordinator,,"Data Analyst (Python), Visualization Specialis...",,Data Scientist/Statistician,
Adejuwon Adegite,,"Technical Writer, Publicity Coordinator",Data Analyst (Python),Publicity Coordinator,Data Analyst (Python),
Kafilat Akinyemi,Visualization Specialist (Power BI),"Data Analyst (Python), Data Scientist/Statisti...",,,"Data Analyst (Python), Publicity Coordinator",Data Scientist/Statistician


Save existing role summary

In [74]:
# current_role_summary.to_csv("role_summary.csv")